In [ ]:
import orsopy
import matplotlib.pyplot as plt

import scipp as sc
import sciline
from ess import amor
from ess.reflectometry.types import *
from ess.amor.utils import ThetaBins, theta_grid

In [ ]:
!git clone https://github.com/jochenstahn/amor.git

In [ ]:
%%bash
python amor/neos.py -Y 2023  -d raw -n 614 -f 614 -mu 0.65

python amor/neos.py -Y 2023  -d raw -n 614 -f 608 -o 608
python amor/neos.py -Y 2023  -d raw -n 614 -f 609 -o 609
python amor/neos.py -Y 2023  -d raw -n 614 -f 610 -o 610
python amor/neos.py -Y 2023  -d raw -n 614 -f 611 -o 611
python amor/neos.py -Y 2023  -d raw -n 614 -f 612 -o 612
python amor/neos.py -Y 2023  -d raw -n 614 -f 613 -o 613

In [ ]:
pl = sciline.Pipeline(
    (*amor.providers, *amor.data.providers, theta_grid),
    params=amor.default_parameters,
)

# Large values disable footprint correction
pl[SampleSize[Sample]] = sc.scalar(10., unit='mm')
pl[SampleSize[Reference]] = sc.scalar(10., unit='mm')

pl[WavelengthBins] = sc.geomspace('wavelength', 2.8, 12, 300, unit='angstrom')
pl[YIndexLimits] =  sc.scalar(11, unit=None), sc.scalar(41, unit=None)
pl[ZIndexLimits] =  sc.scalar(80, unit=None), sc.scalar(370, unit=None)

# The sample rotation value in the file is slightly off, so we set it manually
pl[SampleRotation[Reference]] = sc.scalar(0.65, unit='deg')
pl[PoochFilename[Reference]] = "amor2023n000614.hdf"

from ess.reflectometry import orso
for p in (*orso.providers, *amor.orso.providers):
    pl.insert(p)

pl[orso.OrsoCreator] = orso.OrsoCreator(
    orsopy.fileio.base.Person(
        name='Max Mustermann',
        affiliation='European Spallation Source ERIC',
        contact='max.mustermann@ess.eu',
    )
)
pl[orso.OrsoSample] = orsopy.fileio.data_source.Sample(
    name='Ni / Ti Multilayer',
    model=orsopy.fileio.data_source.SampleModel(
        stack='air | (Ni | Ti) * 5 | Si',
    ),
)

In [ ]:
from ess.reflectometry import save_reference, load_reference

pl[ReferenceFilePath] = save_reference(pl, './reference.hdf5')
pl.insert(load_reference)

In [ ]:
pl[QBins] = sc.geomspace(dim='Q', start=0.00505, stop=2.93164766e-01, num=391, unit='1/angstrom')

for i in range(608, 614):
    print(i, '...', end='')
    pl[PoochFilename[Sample]] = f"amor2023n000{i}.hdf"
    pl.compute(orso.OrsoIofQDataset).save(f'raw/ess-{i}.ort')
    print('Done!')


In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))

for ax, i in zip(axs.ravel(), range(608, 614)):

    d = orsopy.fileio.load_orso(f'raw/{i}.Rqz.ort')[0].data
    ax.errorbar(d[:, 0], d[:, 1], yerr=d[:, 2], label='PSI Amor software')

    d = orsopy.fileio.load_orso(f'raw/ess-{i}.ort')[0].data
    d[d[:, 1] == 0, 1:3] = float('nan')
    ax.errorbar(d[:, 0], d[:, 1], yerr=d[:, 2], label='ESS Reflectometry')
    ax.set_yscale('log')
    ax.set_ylim([1e-4, None])
    ax.legend()
    ax.set_title(f'amor2023n000{i}')
    ax.set_xlabel('Q [1/A]')
    ax.set_ylabel('Reflectivity')

plt.tight_layout()